In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
from posarmctools.readdata import *
%load_ext autoreload
%autoreload 2

In [3]:
import sys

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [9]:
data_date = "2016_09_06_11_38_23"
data_dir = "/home/pleroy/DATA/2016/" + data_date

In [10]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters( params_filename )

In [15]:
blocksPerFile = params.blocksPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "blocksPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    blocksPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 36
nbFiles = 48
lastFile = firstFile + nbFiles - 1
firstBlock = firstFile * blocksPerFile

blocksPerFile = 6, samplesPerRamp = 54000, rampsPerFile = 180


### Read the data

In [16]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

adc_A = np.zeros( (nbFiles, samplesPerFile) )
adc_B = np.zeros( (nbFiles, samplesPerFile) )

In [17]:
lastBlock = (nbFiles-1) * blocksPerFile + firstBlock
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * blocksPerFile + firstBlock) )
    filename = data_dir + "/record" + nb + ".bin"
    readFile( filename, samplesPerFile, timeSerie_A, timeSerie_B )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBlock ))
    
    adc_A[ k, : ] = timeSerie_A
    adc_B[ k, : ] = timeSerie_B

k = 0, block = 216 / 498
k = 1, block = 222 / 498
k = 2, block = 228 / 498
k = 3, block = 234 / 498
k = 4, block = 240 / 498
k = 5, block = 246 / 498
k = 6, block = 252 / 498
k = 7, block = 258 / 498
k = 8, block = 264 / 498
k = 9, block = 270 / 498
k = 10, block = 276 / 498
k = 11, block = 282 / 498
k = 12, block = 288 / 498
k = 13, block = 294 / 498
k = 14, block = 300 / 498
k = 15, block = 306 / 498
k = 16, block = 312 / 498
k = 17, block = 318 / 498
k = 18, block = 324 / 498
k = 19, block = 330 / 498
k = 20, block = 336 / 498
k = 21, block = 342 / 498
k = 22, block = 348 / 498
k = 23, block = 354 / 498
k = 24, block = 360 / 498
k = 25, block = 366 / 498
k = 26, block = 372 / 498
k = 27, block = 378 / 498
k = 28, block = 384 / 498
k = 29, block = 390 / 498
k = 30, block = 396 / 498
k = 31, block = 402 / 498
k = 32, block = 408 / 498
k = 33, block = 414 / 498
k = 34, block = 420 / 498
k = 35, block = 426 / 498
k = 36, block = 432 / 498
k = 37, block = 438 / 498
k = 38, block = 444 / 

In [18]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)
B_reshaped = adc_B.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [19]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

**Set the shifted flag properly**

In [20]:
shifted = 0

In [21]:
ifft_a = np.fft.ifft( (A_reshaped[::,0:3000]), axis = 1 )

In [22]:
plt.figure()
vmin = -15
vmax = 0
nbPointsInRange = 500
plt.plot(  20 * np.log10( np.abs( ifft_a[2500,:] ) ) )
plt.plot( np.ones( nbPointsInRange ) * vmin, "b" )
plt.plot( np.ones( nbPointsInRange ) * vmax, "r" )

In [23]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,:] ) ), 
            aspect='auto', vmin=-15, vmax=0, cmap='jet' )
plt.title("Track 1 - record 2017_10_12_11_47_05\nFiles {} to {}".format(firstFile, lastFile) )

## Remove coupling from raw data

In [40]:
coupling_A = np.average( A_reshaped, 0 )
A2 = A_reshaped - coupling_A

## Build RD

In [24]:
RDc = build_rd_from_data( params, A_reshaped, shifted )

shape of the samples matrix = (8640, 54000)
Data are NOT shifted


In [25]:
coupling_RD = np.average(RDc, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD )

In [28]:
RD2 = RDc - coupling_RD

In [29]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [26]:
np.save(data_dir + '/RD_files_{}_{}'.format(firstFile, lastFile), RDc)

In [27]:
ifft_RD = np.fft.ifft(RDc, axis = 1)
print('ifft_RD done')

ifft_RD done


In [30]:
nbPointsInRange = 300
x0 = 0
x1 = x0 + nbPointsInRange
y0 = 0
y1 = 0
extent = [ x0, x1, y0, y1 ]
rangeStart = 0
rangeStop = 500
azStart = 6000
azStop = 15000

In [31]:
plt.matshow( 20 * np.log10( np.abs( ifft_RD[:, rangeStart:rangeStop] ) ), aspect='auto')
plt.title(data_date)
ax = plt.gca()
plt.colorbar()

In [32]:
fft2d = np.fft.fftshift( np.fft.fft( ifft_RD, axis = 0 ) , axes = 0)
print('fft2d done')

fft2d done


In [33]:
plt.matshow( 20 * np.log( np.abs( fft2d[:, rangeStart:rangeStop] ) ), aspect='auto', cmap='jet', vmin=-60, vmax=50)
plt.title(data_date)
ax = plt.gca()
plt.colorbar()

In [35]:
adc_A_all = adc_A.reshape( rampsPerFile * nbFiles * samplesPerRamp )

In [39]:
plt.figure()
ramp = 10
shift = 38
sampleStart = ramp * samplesPerRamp
plt.plot( adc_A_all[sampleStart : sampleStart+6000 ], 'k',
         label='both ramps' )
plt.plot( adc_A_all[sampleStart : sampleStart+3000 ],
         label='up ramp' )
plt.plot( np.flipud( adc_A_all[ sampleStart+3000 : sampleStart+6000 ] ),
         label='flipped down ramp' )
plt.plot( np.flipud( adc_A_all[ (sampleStart+3000+shift) : (sampleStart+6000+shift) ] ),
         label='flipped down ramp SHIFTED' )
plt.grid()
plt.legend()

In [43]:
plt.figure()
ramp = 0
plt.plot( A2[ramp, 0 : 6000 ], 'k', label='both ramps' )
plt.plot( A2[ramp, 0 : 3000 ], label='up ramp' )
plt.plot( np.flipud( A2[ramp, 3000 : 6000 ] ), label='flipped down ramp' )
plt.grid()
plt.legend()
plt.title('A2')

In [42]:
params.samplesPerRamp

54000

In [ ]:
shifted

In [ ]:
import scipy.signal as signal

In [ ]:
s1 = A_reshaped[ramp, 0 : 3000 ]
s2 = np.flipud( A_reshaped[ramp, 4000 : 5500 ] )

corr = signal.correlate( s1, s2, mode='valid' )

plt.figure()

plt.subplot(211)
plt.plot( corr, '.' )
plt.grid()

plt.subplot(212)
plt.plot( s1 )
plt.plot( s2 )
plt.plot( np.arange(1500) + 538, s2 )
plt.grid()

plt.suptitle( 'max = {}'.format( np.where( corr == np.max(corr) )[0] ) )

In [ ]:
plt.plot(s1)